In [71]:
from model import MiniCharGPTLM
from tokenizer import CharTokenizer
import torch

model_path = "./model/model-hispanic.pth"
ckpt = torch.load(model_path)

tokenizer = CharTokenizer()
model = MiniCharGPTLM(h_dim=ckpt["h_dim"],
                      ff_dim=ckpt["ff_dim"],
                      n_head=ckpt["n_head"],
                      n_block=ckpt["n_block"],
                      n_token=len(tokenizer.char2id))

model.load_state_dict(ckpt["state_dict"])

<All keys matched successfully>

In [77]:
import random
from data import LanguageModelingDataCollator
from string import ascii_lowercase

collator = LanguageModelingDataCollator(tokenizer=tokenizer)

length = 32
k = 3
text = random.choice(ascii_lowercase)
for l in range(length):
    inputs = [tokenizer(text, truncate=True, padding=True, max_length=length)]

    inputs = collator(inputs) # right shift, remove end of token, begin with padding token

    inputs.pop("labels")

    model.eval()
    with torch.no_grad():
        out = model.forward(**inputs)[-1][-1]
    
    # top k sampling
    out = torch.topk(out.cpu(), k).indices
    out = random.choice(out).item()
    next_char = tokenizer.id2char[out]
    if next_char == tokenizer.end_token:
        break
    text += next_char
print(text)

orleote rioda
